# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import holoviews as hv
from holoviews import opts
#import hvplot.pandas

from src import CTX, FOLDERS
from src.data import io
from src.features import COLUMNS, build_features

In [ ]:
hv.extension('bokeh', 'matplotlib')

# Read data

In [ ]:
sales, test, items, categories, shops = io.read_raw(FOLDERS.RAW)

In [ ]:
print('Sales shape: ', sales.shape)
print('Test shape: ', test.shape)
print('Items shape: ', items.shape)
print('Categories shape: ', categories.shape)
print('Shops shape: ', shops.shape)

In [ ]:
observations = []

# Items

In [ ]:
items.head()

In [ ]:
items.nunique()

In [ ]:
items.info()

In [ ]:
(hv.Scatter(items.reset_index(), kdims=['index', 'item_id']) + hv.HexTiles(items, kdims=['item_id', 'item_category_id']).opts(width=500, colorbar=True))

In [ ]:
items_by_category = items[['item_id', 'item_category_id']].groupby('item_category_id').count()
hv.Bars(items_by_category).opts(width=1200, logy=True, ylabel='Item Count (log)')

In [ ]:
observations += ['Tens of categories with very few items, a handfuld of categories with thousands of items.']

# Categories

In [ ]:
categories.head()

In [ ]:
categories.nunique()

In [ ]:
categories.info()

In [ ]:
hv.Scatter(categories.reset_index(), kdims=['index', 'item_category_id']) + hv.Table(categories.sort_values(by='item_category_name'))

In [ ]:
observations += ['First word of category name a good candidate for super-category grouping of categories (split on dash and space)'] 
observations += ['(Categories already sorted on category name)'] 

In [ ]:
categories = build_features.add_super_category(categories)
hv.Table(categories)

# Shops

In [ ]:
shops.head()

In [ ]:
shops.nunique()

In [ ]:
shops.info()

In [ ]:
hv.Scatter(shops.reset_index(), kdims=['index', 'shop_id']) + hv.Table(shops.sort_values(by='shop_name'))

In [ ]:
observations += ['First word of shop name a good candidate for super-category grouping of shops (split on dash and space)'] 
observations += ['(Shops already sorted on shop name)'] 

In [ ]:
shops = build_features.add_super_shop(shops)
hv.Table(shops)

# Sales data

In [ ]:
sales.head()

In [ ]:
observations += ['Submissions are evaluated by root mean squared error (RMSE). True target values are clipped into [0,20] range.']

In [ ]:
sales_rolled_up = build_features.rollup_and_clip_sales(sales)
sales_rolled_up.head()

In [ ]:
train = build_features.enrich(sales_rolled_up, shops, items, categories)
train.head()

In [ ]:
train.nunique()

In [ ]:
train.info()

In [ ]:
observations += ['All shop, item, and category references are valid in training data.']
observations += ['Training data does not have explicit NULLs']
observations += ['All shops and categories, and most items appear in training data']

In [ ]:
plots = [hv.Scatter(train.sample(frac=0.1).reset_index(), kdims=['index', feature_name]).opts(width=500) \
         for feature_name in ['date_block_num', 'shop_id', 'super_shop_id', 'item_category_id', 'super_category_id', 'item_id']]
hv.Layout(plots).cols(2)

In [ ]:
observations += ['My rollup and enrichment code leaves data sorted by time, shop, category, so train/validation split for CV will use different suffix lengths of the data for validation.']

In [ ]:
hv.Bars(train.item_cnt_month.value_counts()).opts(width=1200, logx=True, xrotation=90, xlabel='item_cnt_month', ylabel='value_count (log)')

In [ ]:
observations += ['The vast majority of items only sell once a month across all shops.']

In [ ]:
hv.Dataset(train.sample(frac=0.1), kdims=['super_shop_id', 'date_block_num'], vdims=['item_cnt_month']).sort().to(
    hv.Violin, kdims=['date_block_num']).opts(width=1200, height=800, title='Violin of sales pr. month for selected super_shop_id (use widget)', toolbar='above')#.layout('super_shop_id').cols(1)

In [ ]:
hv.Dataset(train.sample(frac=0.1), kdims=['super_category_id', 'date_block_num'], vdims=['item_cnt_month']).sort().to(
    hv.Violin, kdims=['date_block_num']).opts(width=1200, height=800, title='Violin of sales pr. month for selected super_category_id (use widget)', toolbar='above')#.layout('super_shop_id').cols(1)

In [ ]:
observations += ['Very long sales tails pr. month (rhyme inteded) for both super_shop_id and super_category_id']

In [ ]:
buf = train.groupby(['super_shop_id', 'super_category_id']).aggregate({'item_cnt_month':['count']}) \
    .sort_values(('item_cnt_month', 'count')).tail(10).reset_index() \
    .merge(train, on=['super_shop_id', 'super_category_id'], how='left')
#buf.head()

In [ ]:
hv.Dataset(buf, kdims=['super_shop_id', 'super_category_id', 'date_block_num'], vdims=['item_cnt_month']) \
    .sort().to(hv.Violin, kdims=['date_block_num']) \
    .opts(width=1200, height=800, title='Violin of sales pr. month for selected super_shop_id and super_category_id (use widgets)', toolbar='above')#.layout('super_shop_id').cols(1)

# Test data

In [ ]:
test.head()

In [ ]:
observations += ['For each ID in the test set (shop_id/item_id combination), one must predict a total number of sales (item_cnt_month) for that date_block_num (34).']

In [ ]:
test = build_features.enrich(test, shops, items, categories)
test.head()

In [ ]:
test_date_block_num = train.date_block_num.max() + 1

# Similarity of key distribution across Train and Test 

In [ ]:
def group_on_keys_and_normalize_counts(df, origin):
    from sklearn.preprocessing import MinMaxScaler
    
    result = df.groupby(['shop_id', 'item_category_id'])['date_block_num'].aggregate({'date_block_num':'count'}).reset_index()
    result = result.assign(normalized_count = MinMaxScaler(feature_range=(0, 100), copy=True).fit_transform(result[['date_block_num']]))
    result = result.drop(columns=['date_block_num'])
    result = result.assign(origin=origin)
    return result    

In [ ]:
ds = hv.Dataset(
    pd.concat([
        group_on_keys_and_normalize_counts(train, 'Train'),
        group_on_keys_and_normalize_counts(test.assign(date_block_num=test_date_block_num).drop(columns=['ID']), 'Test')],
        ignore_index=True, sort=False),
    ['shop_id', 'item_category_id', 'origin'], ['normalized_count'])
ds.select(origin=['Train','Test']).to(hv.HeatMap).opts(opts.HeatMap(width=700, height=700, colorbar=True)).grid('origin')

In [ ]:
observations += ['Test evenly distributed, Train more random, will have to explicitly add missing zeroes to Train.']

# Visualizations

In [ ]:
def nansum(a, **kwargs):
    return np.nan if np.isnan(a).all() else np.nansum(a, **kwargs)

## Sales pr. time and shop heatmap

In [ ]:
train.hvplot.heatmap('date_block_num', 'shop_name', 'item_cnt_month', 
                     reduce_function=nansum,
                     title='item_cnt_month pr. date_block_num and shop_name',
                     logz=True, height=800, width=1200, flip_yaxis=True)

In [ ]:
train.hvplot.heatmap('date_block_num', 'super_shop_name', 'item_cnt_month', 
                     reduce_function=nansum,
                     title='item_cnt_month pr. date_block_num and super_shop_name',
                     logz=True, height=800, width=1200, flip_yaxis=True)

## Sales pr time and category heatmap

In [ ]:
train.hvplot.heatmap('date_block_num', 'item_category_name', 'item_cnt_month', 
                     reduce_function=nansum,
                     title='item_cnt_month pr. date_block_num and item_category_name',
                     logz=True, height=800, width=1200, flip_yaxis=True)

In [ ]:
train.hvplot.heatmap('date_block_num', 'super_category_name', 'item_cnt_month', 
                     reduce_function=nansum,
                     title='item_cnt_month pr. date_block_num and super_category_name',
                     logz=True, height=800, width=1200, flip_yaxis=True)

# Observations

In [ ]:
io.save_data(FOLDERS.PROCESSED, CTX + 'observations', pd.DataFrame(observations, columns=['Observation']))